In [34]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [35]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,input):
        input = F.max_pool2d(F.relu(self.conv1(input)),(2,2))
        input = F.max_pool2d(F.relu(self.conv2(input)),(2,2))
        print(input.shape)
        input = torch.flatten(input,1)
        input = F.relu(self.fc1(input))
        input = F.relu(self.fc2(input))
        x = self.fc3(input)
        return x


In [36]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [37]:
# 模型的可学参数
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [38]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)
print(out.argmax(1).item())
writer = SummaryWriter("nn_net") 
writer.add_graph(net,input)
writer.close()

torch.Size([1, 16, 5, 5])
tensor([[ 0.0179,  0.0200,  0.0365, -0.1071,  0.0356, -0.1751,  0.0623,  0.0468,
         -0.1521, -0.0769]], grad_fn=<AddmmBackward0>)
6
torch.Size([1, 16, 5, 5])
torch.Size([1, 16, 5, 5])
torch.Size([1, 16, 5, 5])


In [39]:
# 使用随机将所有参数和反向道具的梯度缓冲区归零 梯度
net.zero_grad()
out.backward(torch.randn(1,10))

In [40]:
# 上面和下面是分开的
# 损失函数

In [41]:

output = net(input)
target = torch.randn(10)  # a dummy target, for example
print(target)
print(target.argmax().item())
# print(target.shape)
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)
print(target.shape)

torch.Size([1, 16, 5, 5])
tensor([ 0.1867, -0.0144,  1.4226, -1.3227,  1.1750,  0.6694, -1.4670, -0.4977,
         0.1439, -1.5149])
2
tensor(1.0230, grad_fn=<MseLossBackward0>)
torch.Size([1, 10])


In [42]:
# 向后走几个步骤
print(loss.grad_fn)# MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [43]:
# 反向传播
# 第一步，清空已有的梯度
net.zero_grad()
print('conv1.bias.grad before backward') # 反向传播之前，第一层卷积偏置的梯度
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad) # 反向传播之后，第一层卷积偏置的梯度

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0070,  0.0111, -0.0127,  0.0056,  0.0381, -0.0006])


In [ ]:
# 更新权重
#weight = weight - learning_rate * gradient
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
"""
for f in net.parameters():：遍历神经网络 net 中的所有参数。神经网络的参数包括权重和偏置。

f.data：获取参数 f 的数据值，即参数的当前值。

f.grad.data：获取参数 f 的梯度值。在反向传播过程中，PyTorch 会自动计算每个参数的梯度，并存储在 grad 属性中。

f.data.sub_(f.grad.data * learning_rate)：这是参数更新的核心部分。它表示将参数的当前值减去梯度乘以学习率的值。这里使用了 PyTorch 中的 sub_() 方法，它表示就地减法操作，即直接修改参数的值而不创建新的张量。
"""

In [50]:
# 反向传播加权重更新
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

#in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

torch.Size([1, 16, 5, 5])
